This is an auxiliary notebook to make the definition of each model easier. With it, models can be defined and saved for later training. 

In [2]:
import torch
import numpy as np
import os
import torch.nn as nn

from torchvision.datasets import ImageFolder
from torchvision.transforms import ToTensor

In [3]:
data_dir = './datasets/Cancer Detection/'
img_dir = data_dir+'labeled images/'
dataset = ImageFolder(img_dir, transform=ToTensor())
test_image = dataset[0][0].unsqueeze(0)

We start by defining the model class in the same way that we did in the training notebooks

In [4]:
class myImageClassification(nn.Module):
    def training_step(self, batch):
        """Takes in a batch of training images and returns their loss"""
        images, labels = batch
        out = self(images)
        loss = F.cross_entropy(out, labels)
        return loss
    
    def validation_step(self, batch):
        """Takes in a batch of validation images and returns their loss and accuracy"""
        images, labels = batch
        out = self(images)
        loss = F.cross_entropy(out, labels)
        acc = accuracy(out, labels)
        tp, tn, fp, fn = confusion(out, labels)
        return {'val_loss': loss.detach(), 'val_acc':acc, 'tp':tp, 'tn':tn, 'fp':fp, 'fn':fn} #detach so its gradient won't be calculated, so not taking up vram
    
    def validation_epoch_end(self, outputs):
        """Takes in the outputs of the validation step and returns the average loss/accuracy for all batches"""
        batch_losses = [x['val_loss'] for x in outputs] #list of validation losses of each batch
        epoch_loss = torch.stack(batch_losses).mean()   #average validation loss for all the batches 
        batch_accs = [x['val_acc'] for x in outputs]    #list of validation accuracy of each batch
        epoch_acc = torch.stack(batch_accs).mean()      #average validation accuracy for all the batches
        epoch_tp = reduce(lambda a,b: a+b,[x['tp'] for x in outputs])
        epoch_tn = reduce(lambda a,b: a+b,[x['tn'] for x in outputs])
        epoch_fp = reduce(lambda a,b: a+b,[x['fp'] for x in outputs])
        epoch_fn = reduce(lambda a,b: a+b,[x['fn'] for x in outputs])
        return {'val_loss':epoch_loss.item(), 'val_acc':epoch_acc.item(), 'tp':epoch_tp, 'tn':epoch_tn, 'fp':epoch_fp, 'fn':epoch_fn}
    
    def epoch_end(self, epoch, result):
        """Prints the result of an epoch"""
        print("\033[94mEpoch [{}]:\033[0m\ntrain_loss: {:.4f}, val_loss: {:.4f} val_acc: {:.4f}".format(
            epoch, result['train_loss'], result['val_loss'], result['val_acc']))
        print(print_confusion(result['tp'], result['tn'], result['fp'], result['fn']))
        print(report(result['tp'], result['tn'], result['fp'], result['fn']),'\n')
        
def accuracy(outputs, labels):
    """Prints the accuracy"""
    _, preds = torch.max(outputs, dim=1)
    return torch.tensor(torch.sum(preds == labels).item() / len(preds))
    

I used auxiliary models to check the output size of layers to make sure that the input of the next ones are set correctly. This can of course also be easily calculated by taking into account the amount of zero padding in convolutions and the size of the pooling kernel, but was especially useful when working without zero padding.

In [5]:
class aux_model(myImageClassification):
    def __init__(self):
        """Define the model's network architecture"""
        super().__init__()
        self.network = nn.Sequential(
            nn.Conv2d(3,21,kernel_size=3,padding=1),
            nn.ReLU(),
#             nn.Conv2d(6,11,kernel_size=3,padding=1),
#             nn.ReLU(),
#             nn.Conv2d(11,17,kernel_size=3,padding=1),
#             nn.ReLU(),
            nn.MaxPool2d(2,2),
            
            nn.Conv2d(21,43,kernel_size=3,padding=1),
            nn.ReLU(),
#             nn.Conv2d(23,28,kernel_size=3,padding=1),
#             nn.ReLU(),
#             nn.Conv2d(28,34,kernel_size=3,padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2,2),
            
            nn.Conv2d(43,64,kernel_size=3,padding=1),
            nn.ReLU(),
#             nn.Conv2d(40,46,kernel_size=3,padding=1),
#             nn.ReLU(),
#             nn.Conv2d(46,51,kernel_size=3,padding=1),
#             nn.ReLU(),
            nn.MaxPool2d(2,2),
        )
        
    def forward(self, xb):
        """Apply the network to the batch of 'x'(images)"""
        return self.network(xb)

size_model = aux_model()
print('Shape of output:', size_model(test_image).shape)

Shape of output: torch.Size([1, 64, 12, 12])


The architectures are defined in the below cell by modifying the code inside the nn.Sequential's parentheses and running the cell, after which the architecture is saved in the form of an untrained model.

In [36]:
class PCamModel(myImageClassification):
    def __init__(self):
        """Define the model's network architecture"""
        super().__init__()
        self.network = nn.Sequential(
            nn.Conv2d(3,14,kernel_size=3,padding=1),
            nn.ReLU(),
            nn.Conv2d(14,14,kernel_size=3,padding=1),
            nn.ReLU(),
            nn.Conv2d(14,14,kernel_size=3,padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2,2),
            
            nn.Conv2d(14,14,kernel_size=3,padding=1),
            nn.ReLU(),
            nn.Conv2d(14,14,kernel_size=3,padding=1),
            nn.ReLU(),
            nn.Conv2d(14,14,kernel_size=3,padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2,2),
            
            nn.Conv2d(14,14,kernel_size=3,padding=1),
            nn.ReLU(),
            nn.Conv2d(14,14,kernel_size=3,padding=1),
            nn.ReLU(),
            nn.Conv2d(14,14,kernel_size=3,padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2,2),
            
            nn.Flatten(),
            nn.Linear(14*12*12, 2),
            nn.Softmax(dim=1),
#             nn.ReLU(),
#             nn.Linear(2048, 256),
#             nn.ReLU(),
#             nn.Linear(256,2)       
        )
        
    def forward(self, xb):
        """Apply the network to the batch of 'x'(images)"""
        return self.network(xb)

    
#INSTANTIATE AND SAVE THE MODEL
group = 'flat'
folder = './Models/'+group+'/'
filename = group+'-128-9'
model = PCamModel()
print(model)
print('Shape of output:', size_model(test_image).shape)
with open(folder+filename+'.txt', 'w') as info_file:
    info_file.write(str(model))
torch.save(model, folder+filename+'.pth')

PCamModel(
  (network): Sequential(
    (0): Conv2d(3, 14, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU()
    (2): Conv2d(14, 14, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU()
    (4): Conv2d(14, 14, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (5): ReLU()
    (6): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (7): Conv2d(14, 14, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): ReLU()
    (9): Conv2d(14, 14, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (10): ReLU()
    (11): Conv2d(14, 14, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (12): ReLU()
    (13): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (14): Conv2d(14, 14, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (15): ReLU()
    (16): Conv2d(14, 14, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (17): ReLU()
    (18): Conv2d(14, 14, kernel_size=(3, 3), stride=(1, 1), p